In [1]:
import numpy as np 
#import sklearn
import matplotlib.pyplot as plt
import scipy.io as io
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC


np.random.seed(1242)

In [7]:
NosOfComponents = np.linspace(1,1,1,dtype='int')
NosImages = 2400
Methods = ['LinearSVM']# choose between 'LinearSVM','PolySVM','QDA','LDA'
Kfold = 5

In [8]:
def split(data, labels, val_size):
    num_items = len(data)
    np.random.seed(1242)
    assert num_items == len(labels)
    assert val_size >= 0
    if val_size < 1.0:
        val_size = int(num_items * val_size)
    train_size = num_items - val_size
    idx = np.random.permutation(num_items)
    data_train1 = data[idx][:train_size]
    label_train = labels[idx][:train_size]
    data_val = data[idx][train_size:]
    label_val = labels[idx][train_size:]
    return data_train1, data_val, label_train, label_val




In [9]:
def KfoldSplit(TrainingData,TrainingLabel,Kfold):
    U,V= TrainingData.shape
    slicelength  =int(np.floor(U/Kfold))
    slicesetdata = np.zeros((Kfold,slicelength,V))
    slicesetlabel = np.zeros((Kfold,slicelength,1),dtype = int)



    for i in range(Kfold):
        slicesetdata[i][:][:]= TrainingData[slicelength*i:slicelength*(i+1)][:].reshape(1,slicelength,V)
        slicesetlabel[i][:][:]= TrainingLabel[slicelength*i:slicelength*(i+1)][:].reshape(1,slicelength,1)
    return slicesetdata,slicesetlabel

In [10]:
def classify(DataVector,Method,Kfold,Cval,IterNos):
    print(Method)
    Labels = io.loadmat('Labels.mat')
    Labels = Labels['Labels']

    U,_ = DataVector.shape
    TestSize = int(0.2*U)

    TrainingData,TestData, TrainingLabel, TestLabel = split(DataVector,Labels,TestSize)
    M = np.max(np.max(np.max((np.array(TrainingData)))))
    TrainingData = TrainingData/M # normalizing the data
    TestData = TestData/M
    ###########################################
    slicesetdata,slicesetlabel = KfoldSplit(TrainingData,TrainingLabel,Kfold)
    U,V = TrainingData.shape
    slicelength  =int(np.floor(U/Kfold))
    ###############################
    
    ValidationAccuracy =np.zeros((IterNos,1), dtype = "float64")
    TestAccuracy =np.zeros((IterNos,1), dtype = "float64")
    for m in range(len(Cval)):
        if (Method =='LDA'):
            clf =  LDA(solver = 'lsqr', shrinkage = Cval[m])
        elif (Method =='QDA'):
            clf =  QDA(reg_param = Cval[m])
        elif (Method =='PolySVM'):
            clf =  SVC(C=Cval[m],kernel ='poly',max_iter=10000,gamma='auto')
        else:
            clf =  SVC(C=Cval[m],kernel ='linear',max_iter=10000,gamma='auto')
        ValidationAccuracyT =0.0
        for i in range(Kfold):
            for j in range (Kfold):
                if j!=i:
                    clf.fit((slicesetdata[j][:][:].reshape(slicelength,V)),(slicesetlabel[j]
                                                                     [:].reshape(slicelength,1)).ravel())
            Check = clf.predict((slicesetdata[i][:][:].reshape(slicelength,V)))
            ValidationAccuracyT= ValidationAccuracyT+(accuracy_score(slicesetlabel[i][:].reshape(slicelength,1), Check))
        ValidationAccuracy[m] = (ValidationAccuracyT/Kfold)
        TestCheck = clf.predict(TestData.reshape(TestSize,V))
        TestAccuracy[m] = (accuracy_score(TestLabel.reshape(TestSize,1), TestCheck))
    return Cval,ValidationAccuracy,TestAccuracy

In [11]:
Results ={}
DataToLoad = io.loadmat('Data1')
DataToLoad = DataToLoad['Data']
U=114
V = 122

Data = np.zeros((NosImages,U*V))
for k in range(NosImages):
    X = DataToLoad[k,:,:].reshape(U,V)
    Data[k,:] = np.array(X.flatten())
Data = Data/np.max(np.max(Data))
##################doing the PCA####################################
for i in range(0,len(NosOfComponents)):
    print('PCA',NosOfComponents[i])
    DataToSave = np.zeros((NosImages,NosOfComponents[i]))
    #######################################

    pca = PCA(n_components=NosOfComponents[i])
    DataToSave = (np.array(pca.fit_transform(Data)))
    #################################running each algorithm############################
    IterNos =150
    Kfold =5
    for j in range(0,len(Methods)):
        if (Methods[j] =='LDA'):
            Cval = np.linspace(1e-5,1,IterNos)
        elif (Methods[j] =='QDA'):
            Cval =  np.linspace(1e-5,1,IterNos)
        elif (Methods[j] =='PolySVM'):
            Cval =  np.logspace(-9,9,IterNos)
        else:
            Cval =  np.logspace(-9,9,IterNos)
        Hyperparameter,TrainingAccuracy,ValidationAccuracy = classify(DataToSave,Methods[j],Kfold,Cval,IterNos)
        Results.update({Methods[j]+str(NosOfComponents[i])+' Method':Methods[j],
                        Methods[j]+str(NosOfComponents[i])+' Training Accuracy': TrainingAccuracy,
                        Methods[j]+str(NosOfComponents[i])+'Validation Accuracy': ValidationAccuracy,
                       Methods[j]+str(NosOfComponents[i])+' Hyperparameter': Hyperparameter,
                       Methods[j]+str(NosOfComponents[i])+' Maximum Validation Accuracy': np.max(ValidationAccuracy)})
        io.savemat('TimeAndFrequency',Results)

PCA 1
LinearSVM


d:\program files\python\python37\lib\site-packages\sklearn\svm\base.py:244: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
d:\program files\python\python37\lib\site-packages\sklearn\svm\base.py:244: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
d:\program files\python\python37\lib\site-packages\sklearn\svm\base.py:244: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
d:\program files\python\python37\lib\site-packages\sklearn\svm\base.py:244: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
d:\p

d:\program files\python\python37\lib\site-packages\sklearn\svm\base.py:244: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
d:\program files\python\python37\lib\site-packages\sklearn\svm\base.py:244: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
d:\program files\python\python37\lib\site-packages\sklearn\svm\base.py:244: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
d:\program files\python\python37\lib\site-packages\sklearn\svm\base.py:244: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
d:\p

d:\program files\python\python37\lib\site-packages\sklearn\svm\base.py:244: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
d:\program files\python\python37\lib\site-packages\sklearn\svm\base.py:244: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
d:\program files\python\python37\lib\site-packages\sklearn\svm\base.py:244: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
d:\program files\python\python37\lib\site-packages\sklearn\svm\base.py:244: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
d:\p

d:\program files\python\python37\lib\site-packages\sklearn\svm\base.py:244: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
d:\program files\python\python37\lib\site-packages\sklearn\svm\base.py:244: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
d:\program files\python\python37\lib\site-packages\sklearn\svm\base.py:244: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
d:\program files\python\python37\lib\site-packages\sklearn\svm\base.py:244: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
d:\p

...

d:\program files\python\python37\lib\site-packages\sklearn\svm\base.py:244: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
d:\program files\python\python37\lib\site-packages\sklearn\svm\base.py:244: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
d:\program files\python\python37\lib\site-packages\sklearn\svm\base.py:244: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
d:\program files\python\python37\lib\site-packages\sklearn\svm\base.py:244: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
d:\p

d:\program files\python\python37\lib\site-packages\sklearn\svm\base.py:244: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
d:\program files\python\python37\lib\site-packages\sklearn\svm\base.py:244: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
d:\program files\python\python37\lib\site-packages\sklearn\svm\base.py:244: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
d:\program files\python\python37\lib\site-packages\sklearn\svm\base.py:244: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
d:\p

d:\program files\python\python37\lib\site-packages\sklearn\svm\base.py:244: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
d:\program files\python\python37\lib\site-packages\sklearn\svm\base.py:244: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
d:\program files\python\python37\lib\site-packages\sklearn\svm\base.py:244: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
d:\program files\python\python37\lib\site-packages\sklearn\svm\base.py:244: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
d:\p

d:\program files\python\python37\lib\site-packages\sklearn\svm\base.py:244: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
d:\program files\python\python37\lib\site-packages\sklearn\svm\base.py:244: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
d:\program files\python\python37\lib\site-packages\sklearn\svm\base.py:244: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
d:\program files\python\python37\lib\site-packages\sklearn\svm\base.py:244: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
d:\p

d:\program files\python\python37\lib\site-packages\sklearn\svm\base.py:244: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
d:\program files\python\python37\lib\site-packages\sklearn\svm\base.py:244: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
d:\program files\python\python37\lib\site-packages\sklearn\svm\base.py:244: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
d:\program files\python\python37\lib\site-packages\sklearn\svm\base.py:244: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
d:\p

In [ ]:
for i in range(0,len(NosOfComponents)):
    for j in range(0,len(Methods)):
        print(Results[Methods[j]+str(NosOfComponents[i])+' Maximum Validation Accuracy'])